# 🦀  Aiven Setup



The setup starts an **Apache Kafke** and a **postgreSQL** database used in the other notebooks
To execute all the steps on top of Aiven.io instances, please register on the [console](https://console.aiven.io?utm_source=github&utm_medium=organic&utm_campaign=blog_art&utm_content=post) 

---

## Set the Environment 

### Set Variables

The following global variables will define the kafka instance, change with care

In [ ]:
%%bash
mkdir -p config
echo "export FOLDER_NAME=kafkacerts
export PROJECT_NAME=my-test-name
export CLOUD=google-europe-west3
export KAFKA_NAME=kafka-pywebconf
export POSTGRES_NAME=pg-pywebconf
export AIVEN_PLAN_NAME=business-4
export TOPIC_NAME=pizzas
export PG_USER=new_pg_user
export PG_PWD=NewPWD123
export KAFKA_TIMEOUT=5000" > config/profile_info.sh

echo "" > config/__init__.py

### Create an Aiven Kafka environment

For this demo we'll create a Kafka environment in [Aiven.io](https://aiven.io/) in order to follow these steps you would need to have a valid login to Aiven.io console and have a token already generated. In the below script change the `<EMAIL>` and `<TOKEN>` fields to match yours. Let's login

In [ ]:
%%bash 
mkdir -p ~/.config/aiven
echo '{
    "auth_token": "<INSERT_TOKEN_HERE>",
    "user_email": "<INSERT_EMAIL_HERE>"
}' > ~/.config/aiven/aiven-credentials.json

pip install aiven-client

After the login we can create our kafka instance

In [ ]:
%%bash

source config/profile_info.sh

avn service create  -p $AIVEN_PLAN_NAME \
                    -t kafka $KAFKA_NAME \
                    --cloud $CLOUD \
                    --project $PROJECT_NAME \
                    -c kafka_rest=true \
                    -c kafka.auto_create_topics_enable=true \
                    -c schema_registry=true \
                    -c kafka_connect=true

avn service create $POSTGRES_NAME -t pg -p startup-4 --cloud $CLOUD --project $PROJECT_NAME

avn service wait $KAFKA_NAME --project $PROJECT_NAME

Get Kafka certificates and service URI. We'll use the certificates to connect

In [ ]:
%%bash
# Download all certificates
source config/profile_info.sh

mkdir -p kafkacerts
avn service user-creds-download $KAFKA_NAME --project $PROJECT_NAME -d $FOLDER_NAME --username avnadmin

# get KAFKA URL and write to disk as `HOSTNAME and PORT`
avn service get $KAFKA_NAME --project=$PROJECT_NAME --format '{service_uri}' | awk  -F':' '{print "hostname = \""$1"\"\nport = "$2"\ncert_folder = \""FOLDER_NAME"\"\ntopic_name = \""TOPIC_NAME"\"\npg_user = \""PG_USER"\"\npg_pwd = \""PG_PWD"\"\ntimeout_ms = "KAFKA_TIMEOUT""}' FOLDER_NAME=$FOLDER_NAME TOPIC_NAME=$TOPIC_NAME PG_USER=$PG_USER PG_PWD=$PG_PWD KAFKA_TIMEOUT=$KAFKA_TIMEOUT > config/kafka_config.py 

Create **PostgreSQL User**

In [ ]:
%%bash

source config/profile_info.sh
#avn service create $POSTGRES_NAME -t pg -p startup-4 --cloud $CLOUD --project $PROJECT_NAME

avn service wait $POSTGRES_NAME --project $PROJECT_NAME
avn service get $POSTGRES_NAME --project=$PROJECT_NAME --format '{service_uri_params}' > config/pg_config.json
avn service user-create $POSTGRES_NAME --project $PROJECT_NAME --username $PG_USER
avn service user-password-reset $POSTGRES_NAME --project $PROJECT_NAME --username $PG_USER --new-password $PG_PWD



Now let's parse the fields and create the Kafka Connect Connector config file

In [ ]:
import json
from config.kafka_config import *
with open('config/pg_config.json') as json_file:
    data = json.loads(json_file.read().replace("'",'"'))
    pg_dbname = data['dbname']
    pg_host = data['host']
    pg_port = data['port']
    pg_super_user = data['user']
    pg_super_pwd = data['password']

connect_setup = {
    "name": "sink_kafka_pg",
    "connector.class": "io.aiven.connect.jdbc.JdbcSinkConnector",
    "value.converter": "org.apache.kafka.connect.json.JsonConverter",
    "topics.regex": topic_name+"_schema",
    "connection.url": "jdbc:postgresql://"+pg_host+":"+pg_port+"/"+pg_dbname+"?sslmode=require",
    "connection.user": pg_user,
    "connection.password": pg_pwd,
    "auto.create": "true"
}

f = open("config/kafka_connect_setup.txt", "w")
f.write(json.dumps(connect_setup, indent=4, sort_keys=True))
f.close()

Connect to postgress and grant access to newly created user

In [ ]:
%%bash 

pip install psycopg2-binary

In [ ]:
import psycopg2
conn = psycopg2.connect(dbname=pg_dbname,
                        user=pg_super_user,
                        host=pg_host,
                        port=pg_port,
                        password=pg_super_pwd,
                        sslmode='require')
cur = conn.cursor()
cur.execute("GRANT CONNECT ON DATABASE defaultdb TO "+pg_user+";")
cur.execute("GRANT USAGE ON SCHEMA public TO "+pg_user+";")
